# 前置作業

## 安裝相關套件

In [ ]:
!pip install lineTool
!pip install fugle-trade -U
!pip install fugle-realtime

In [15]:
import datetime
import lineTool
import fugle_trade
from fugle_realtime import WebSocketClient
import pandas as pd
import requests
import time
import json
from fugle_trade.constant import *
from fugle_trade.order import OrderObject
from fugle_trade.sdk import SDK
from configparser import ConfigParser

## 取得相關 Token 及輸入設定檔

In [7]:
# Set Your Fugle API Token from https://developer.fugle.tw/docs/key/
API_TOKEN = 'YOUR_API_TOKEN'

# Set Your line Token from https://notify-bot.line.me/my/
LINE_NOTIFY_TOKEN = 'YOUR_LINE_NOTIFY_TOKEN'

In [19]:
# 連結富果 API 設定檔並登入
config = ConfigParser()
config.read('./config.simulation.ini') # 使用模擬環境

# 登入
sdk = SDK(config)
sdk.login()

### 下單測試

In [4]:
# 下單測試
symbol_id = "0050"
qty = 1

order = OrderObject(
    buy_sell=Action.Buy,
    price_flag=PriceFlag.LimitDown,
    price = '',
    stock_no=symbol_id,
    quantity=qty,
    ap_code=APCode.Common,
    trade=Trade.Cash
)
sdk.place_order(order)

{'ord_date': '20221021',
 'ord_time': '1310SS168',
 'ord_type': '2',
 'ord_no': 'Z0241',
 'ret_code': '000000',
 'ret_msg': '',
 'work_date': '20221021'}

# 標的選擇

In [13]:
# 已發行股數 data 
total_data = pd.read_excel('shares_data.xlsx')

In [ ]:
# 週轉率 =  成交量/ 已發行股數，串接歷史 API 方式計算
volume_list = []

for symbol_id in total_data['symbol_id'].tolist():

    url = f"https://api.fugle.tw/marketdata/v0.3/candles?symbolId={symbol_id}&apiToken={API_TOKEN}&from=2022-10-15&to=2022-10-25"
    
    try:
        volume = requests.get(url).json()['candles'][-1]['volume']

        volume_list.append([symbol_id,volume])
    
    except KeyError:
        
        pass
    

In [17]:
# 週轉率排行 -> 這部分如果是 finlab vip 透過 finlab 平台取得這些盤後資料，應該會方便許多

df = pd.merge(pd.DataFrame(volume_list, columns=['symbol_id','volume']),total_data)

df['turnover_rate'] = df['volume']/df['已發行普通股數或TDR原股發行股數']

df.sort_values(['turnover_rate','volume'], ascending=False).head(50)

,symbol_id,volume,已發行普通股數或TDR原股發行股數,turnover_rate
259,4931,61481990,62658761,0.981219
1294,3046,54487109,71448013,0.762612
913,1760,18936240,76739013,0.246762
58,8996,19556567,89384080,0.218793
1114,2468,14434060,69961249,0.206315
1438,3454,17586276,86957393,0.202240
1072,2413,24595039,127359200,0.193116
1284,3035,39053035,248550313,0.157123
1583,6533,6305458,42650911,0.147839
1526,3661,9812510,69890218,0.140399


# 程式碼實作

In [ ]:
# 查看 server 時間 -> 與本地時間差八小時，因此下面的時間判斷需 -8
datetime.datetime.now()

datetime.datetime(2022, 10, 21, 1, 23, 21, 672730)

In [ ]:
# gridTrading_strategy

class GridTrading:

    # 預計設定方便調整的參數有 股票代碼、網格大小的比例 以及初始部位
    def __init__(self, symbol_id, grid_size_ratio, first_position):
        
        # 初始化交易 API 並取得 sdk object
        self.sdk = self._init_fugle_trade()
        
        # 設定交易標的
        self.symbol_id = symbol_id

        # 首筆交易建倉幾張 -> 預計花 25% 的部位建倉
        self.first_position = first_position

        # 網格大小的比例 -> 與交易頻率有關
        self.grid_size_ratio = grid_size_ratio

        # 設定第一次買進的價格 -> 基準價
        self.base_price = None

        # 網格大小
        self.grid_size = None
        # 網格大小的比例 * 基準價 = 網格大小
        
        # 目前價格
        self.now_price = None
        
        # 目前持有部位數量
        self.now_position = 0

    # 交易 API 設定檔的部分
    def _init_fugle_trade(self):
        # 讀取設定檔
        config = ConfigParser()
        config.read('./config.simulation.ini') # 使用模擬環境
        # 登入
        sdk = SDK(config)
        sdk.login()

        return sdk

    # 透過下面的 webSocket  function 取得最新價格的部分
    def _on_new_price(self, message):
        json_data = json.loads(message)

        # 只使用整股最新價格
        if json_data['data']['info']['type'] == "EQUITY":
            # 更新目前價格
            self.now_price = json_data['data']['quote']['trade']['price']

            print(json_data['data']['quote']['trade']['price'])

    def create_ws_quote(self):
        ws_client = WebSocketClient(api_token=API_TOKEN)
        ws = ws_client.intraday.quote(symbolId=self.symbol_id, on_message=self._on_new_price)
        ws.run_async()
        time.sleep(3)

    def buy(self, qty):
        order = OrderObject(
            buy_sell=Action.Buy,
            price_flag=PriceFlag.Market,
            price = '',
            stock_no=self.symbol_id,
            quantity=qty,
            ap_code=APCode.Common,
            trade=Trade.Cash
        )
        #
        self.sdk.place_order(order)
        msg = '\n'+f'目前有 {str(self.now_position)} 張部位'+'\n'+'買進' + self.symbol_id + '\n' + str(qty) + "張" + '\n' + str(self.now_price) +'元'
        lineTool.lineNotify(LINE_NOTIFY_TOKEN, msg)

    def sell(self, qty):
        order = OrderObject(
            buy_sell=Action.Sell,
            price_flag=PriceFlag.Market,
            price = '',
            stock_no=self.symbol_id,
            quantity=qty,
            ap_code=APCode.Common,
            trade=Trade.DayTradingSell   # 現股當沖賣 的交易類別
        )
        #
        self.sdk.place_order(order)
        msg = '\n'+f'目前有 {str(self.now_position)} 張部位'+'\n'+'賣出' + self.symbol_id + '\n' + str(qty) + "張"+'\n'+ str(self.now_price) +'元'
        lineTool.lineNotify(LINE_NOTIFY_TOKEN, msg)

    # 在 13:25 後，如果還有部位時，因為這是當沖策略，所以會把所有部位全部賣出
    # 但因為交易規則的關係，不能以市價賣，須以漲停價賣出

    def sell_allposition(self, qty):
        order = OrderObject(
            buy_sell=Action.Sell,
            price_flag=PriceFlag.LimitUp, # 漲停價賣出
            price = '',
            stock_no=self.symbol_id,
            quantity=qty,
            ap_code=APCode.Common,
            trade=Trade.DayTradingSell   # 現股當沖賣
        )
        
        self.sdk.place_order(order)
        msg = '\n'+f'目前有 {str(self.now_position)} 張部位'+'\n'+'賣出' + self.symbol_id + '\n' + str(qty) + "張"+'\n'+ str(self.now_price) +'元'
        lineTool.lineNotify(LINE_NOTIFY_TOKEN, msg)


    def run_trade(self):
        
        if self.now_price is not None:

            # 設定第一次買進的價格
            self.base_price = self.now_price
            # 設定網格大小
            self.grid_size = self.base_price * self.grid_size_ratio

            # 買進部位
            self.now_position = self.first_position
            self.buy(self.now_position)
        
        # 如果是盤中
        while datetime.time(9, 0, 0) < datetime.datetime.now().time() < datetime.time(13, 25, 0):

            if self.now_position > 0 and self.now_price >= self.base_price+((self.first_position+1 - self.now_position)*self.grid_size):

                # 賣出一部位
                self.sell(1)
                # 更新
                self.now_position = self.now_position - 1

            # 透過部位持有上限來計算下限價格
            elif self.now_position < (self.first_position)*4 and self.now_price <= self.base_price-((self.now_position - (self.first_position-1)) * self.grid_size):

                # 買進一部位
                self.buy(1)
                # 更新
                self.now_position = self.now_position + 1

        # 13:25 最後一盤無法以市價賣出， 需使用漲停價賣出所有部位
        if self.now_position > 0:
            self.sell_allposition(self.now_position)


if __name__ == '__main__':
    # 輸入要交易的標的，以第一筆成交價格的 0.5% 作為網格大小的比例
    gt = GridTrading('6150', 0.005, 2)
    gt.create_ws_quote()
    gt.run_trade()



本程式無法全面考量各種風險 僅做為教學之用<br>
網格也存在可能跌停，賣不出去的風險 ，所以即使是當沖交易也應該要準備好相對應的交割款，才不會違約交割哦！

In [18]:
import pandas as pd
pd.DataFrame(sdk.get_order_results())

,work_date,ord_date,ord_time,ord_status,ord_no,pre_ord_no,stock_no,buy_sell,ap_code,price_flag,...,mat_qty,cel_qty,celable,err_code,err_msg,avg_price,bs_flag,org_qty_share,mat_qty_share,cel_qty_share
0,20221021,20221021,1310SS793,2,Z0245,,6150,S,1,4,...,0,0,2,00000000,,0.0,R,1000,0,0
1,20221021,20221021,1310SS780,2,Z0243,,6150,S,1,4,...,1,0,2,00000000,,0.0,R,1000,1000,0
2,20221021,20221021,1310SS607,2,Z0250,,6150,S,1,4,...,0,0,2,00000000,,0.0,R,2000,0,0
3,20221021,20221021,1310SS526,2,Z0246,,6150,S,1,4,...,1,0,2,00000000,,0.0,R,1000,1000,0
4,20221021,20221021,1310SS324,2,Z0248,,6150,B,1,4,...,2,0,2,00000000,,0.0,R,2000,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,20221021,20221021,0810SS626,2,Z0004,P0E000YB,3046,B,1,4,...,2,0,2,00000000,,0.0,R,2000,2000,0
248,20221021,20221021,0810SS620,2,Z0003,P0E000YA,3046,S,1,4,...,1,0,2,00000000,,0.0,R,2000,1000,0
249,20221021,20221021,0810SS615,2,Z0002,P0E000Y9,3046,B,1,4,...,2,0,2,00000000,,0.0,R,2000,2000,0
250,20221021,20221021,0810SS585,2,Z0001,P0E000Y8,0050,B,1,4,...,1,0,2,00000000,,0.0,R,1000,1000,0
